In [2]:
from os import listdir
from os.path import join
import numpy as np
import scipy as sp
from scipy import stats
import c_writer

In [50]:
# Settings (define file locations)
models_path = 'models'  # Where we can find the model files (relative path location)
md_file_name = 'md_model-apnea'   # Mahalanobis Distance model arrays (.npz will be added)
c_model_name = 'md_model-normal'   # Will be given .h suffix

c_mean_name = 'model_mu'
c_inv_cov_name = 'model_inv_cov'

In [51]:
# Load model
from numpy import load

data = load('models\\normal_apnea_samples.npz')
lst = data.files
for item in lst:
    print(item)
    print(data[item])
    
with np.load(join(models_path, md_file_name) + '.npz') as data:
    model_mu = data['model_mu']
    model_cov = data['model_cov']
print(model_mu)
print(model_cov)

normal_sample
[[-0.042101  0.042501  0.004663]
 [-0.042234  0.038237  0.00493 ]
 [-0.041168  0.034374  0.004663]
 [-0.041035  0.032642  0.005329]
 [-0.039836  0.023848  0.002798]
 [-0.039436  0.019851  0.006395]
 [-0.040769  0.01732   0.005329]
 [-0.041035  0.011325  0.001599]
 [-0.044766  0.007061  0.003597]
 [-0.047031  0.00453   0.003464]
 [-0.050894  0.001199 -0.000666]
 [-0.050095 -0.004263 -0.001865]
 [-0.050894 -0.002265  0.000933]
 [-0.049029 -0.003464 -0.002398]
 [-0.05196  -0.001199 -0.002398]
 [-0.05196   0.003198 -0.002931]
 [-0.052893  0.005596 -0.001066]
 [-0.055691  0.011191 -0.000266]
 [-0.05689   0.016521 -0.001199]
 [-0.059821  0.021983 -0.001599]
 [-0.062485  0.028645 -0.001998]
 [-0.065683  0.027579 -0.003464]
 [-0.06968   0.031043 -0.006928]
 [-0.067149  0.028112 -0.007994]
 [-0.064084  0.02598  -0.006528]
 [-0.055025  0.019851 -0.002798]
 [-0.049162  0.008926 -0.002798]
 [-0.04743   0.001998 -0.005462]
 [-0.040636 -0.003464  0.000666]
 [-0.042368 -0.004263  0.    

In [53]:
# Calculate inverse of covariance matrix (as it's constant)
inv_cov = sp.linalg.inv(model_cov)
print(inv_cov)

[[ 278337.66110181  -22727.85209541 -441112.0503078 ]
 [ -22727.85209541   79568.38898929  -90557.58133399]
 [-441112.0503078   -90557.58133399 1600360.97989409]]


In [54]:
# Create constant C arrays for model (mu and inverse covariance)
c_mu = c_writer.create_array(model_mu, 'float', c_mean_name)
c_inv_cov = c_writer.create_array(inv_cov, 'float', c_inv_cov_name)

In [55]:
# Construct header file
header_str = c_writer.create_header(c_mu + '\n' + c_inv_cov, c_model_name)
print(header_str)

#ifndef MD_MODEL-NORMAL_H
#define MD_MODEL-NORMAL_H

const unsigned int model_mu_dim1 = 3;

const float model_mu[3] = {
    0.007046512684615385, 0.013648587507692307, 0.002951542973076922
};

const unsigned int model_inv_cov_dim1 = 3;
const unsigned int model_inv_cov_dim2 = 3;

const float model_inv_cov[3][3] = {
    278337.66110181215, -22727.852095408543, -441112.0503077989, 
    -22727.852095408547, 79568.3889892913, -90557.58133398659, 
    -441112.05030779896, -90557.58133398659, 1600360.9798940928
};

#endif //MD_MODEL-NORMAL_H


In [56]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Convert raw sample to C array for testing.

In [60]:
# Saved Numpy test samples file location
sample_file_path = 'test_samples'
sample_file_name = 'normal_apnea_samples'  # Will be given .npz suffix
c_normal_sample_name = 'normal_sample'       # Will be given .h suffix for file
c_apnea_sample_name = 'apnea_sample'     # Will be given .h suffix for file

In [61]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    apnea_sample = data['apnea_sample']
print(normal_sample.shape)
print(apnea_sample.shape)
print(normal_sample[:5])
print(apnea_sample[:5])

(200, 3)
(200, 3)
[[-0.06515  -0.010259 -0.005462]
 [-0.063818 -0.014655 -0.004263]
 [-0.062086 -0.009726 -0.003064]
 [-0.059954 -0.01732  -0.001466]
 [-0.057423 -0.01732  -0.001732]]
[[-0.053559 -0.027446 -0.007194]
 [-0.050894 -0.027979 -0.005729]
 [-0.052893 -0.022649 -0.005462]
 [-0.048363 -0.017986 -0.007461]
 [-0.049695 -0.013989 -0.007061]]


In [62]:
# Create C array out of normal sample
c_sample = c_writer.create_array(normal_sample, 'float', c_normal_sample_name)
header_str = c_writer.create_header(c_sample, c_normal_sample_name)
#print(header_str)

In [63]:
# Save C header file with normal sample
with open(join(sample_file_path, c_normal_sample_name) + '.h', 'w') as file:
    file.write(header_str)

In [64]:
# Create C array out of apnea sample
c_sample = c_writer.create_array(apnea_sample, 'float', c_apnea_sample_name)
header_str = c_writer.create_header(c_sample, c_apnea_sample_name)
print(header_str)

#ifndef APNEA_SAMPLE_H
#define APNEA_SAMPLE_H

const unsigned int apnea_sample_dim1 = 200;
const unsigned int apnea_sample_dim2 = 3;

const float apnea_sample[200][3] = {
    -0.053559, -0.027446, -0.007194, -0.050894, -0.027979, -0.005729, 
    -0.052893, -0.022649, -0.005462, -0.048363, -0.017986, -0.007461, 
    -0.049695, -0.013989, -0.007061, -0.050228, -0.003997, -0.007461, 
    -0.050228, -0.003464, -0.006129, -0.04783, 0.000133, -0.007328, -0.048496, 
    0.001466, -0.005196, -0.049296, 0.003198, -0.00493, -0.049828, 0.004263, 
    -0.002531, -0.050095, 0.002398, -0.001865, -0.05196, 0.001732, -0.002265, 
    -0.049162, -0.000533, -0.002132, -0.050361, -0.001865, -0.0004, -0.049962, 
    -0.005329, -0.003997, -0.050628, -0.00453, -0.001865, -0.050095, -0.005862, 
    -0.001865, -0.050228, -0.006528, -0.003331, -0.050095, -0.009992, 0.000533, 
    -0.048763, -0.009326, -0.002132, -0.051694, -0.015188, -0.002265, 
    -0.052626, -0.01359, 0.000133, -0.052493, -0.009459, -0.0004, 

In [65]:
# Save C header file with normal sample
with open(join(sample_file_path, c_apnea_sample_name) + '.h', 'w') as file:
    file.write(header_str)

C functions

In [66]:
# Set some settings to test out our functions
sensor_sample_rate = 200    # Hz
sample_time = 0.64           # Time (sec) length of each sample
max_measurements = int(sample_time * sensor_sample_rate)

In [67]:
# Test extracting features (median absolute deviation) using SciPy
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
sp_test_features = stats.median_absolute_deviation(sample)  # Calculate MAD
print(sp_test_features)

[0.00651825 0.01135746 0.00207342]


C:\Users\beamtana\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
# Find median in an array (we'll use qsort in C)
# We want to calculate MAD in a C-like fashion to make it easier to port
def median(arr):
    n = len(arr)
    sorted_arr = np.sort(arr)
    if n % 2 == 0:
        median = (sorted_arr[int((n - 1) / 2)] + sorted_arr[int(n / 2)]) / 2.0
    else:
        median = sorted_arr[n / 2]
    return median

In [69]:
# Calculate MAD manually
def calc_mad(arr):
    
    # Find number of elements
    n = len(arr)
    
    # Get median of array
    med = median(arr)

    # Calculate absolute deviation from median for each element
    devs = [0] * n
    for i in range(n):
        devs[i] = abs(arr[i] - med)
    
    # Find median of deviations
    return median(devs)

In [70]:
# Wrapper to extract MAD features for all axis
def extract_mad_features(sample, scale=1.4826):
    mads = []
    
    # Calculate MAD for each axis (note: SciPy scales by 1.4826)
    # https://github.com/scipy/scipy/issues/11090
    for axis in range(sample.shape[1]):
        mads.append(scale * calc_mad(sample[:, axis]))
        
    return mads

In [71]:
# Test MAD algorithm and compare to SciPy answer
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = extract_mad_features(sample)
print(normal_x)

[0.006518250900000001, 0.011357457299999998, 0.0020734161]


In [72]:
# Test: calculate Mahalanobis distance using Numpy
x_minus_mu = normal_x - model_mu
left_term = np.dot(x_minus_mu, inv_cov)
mahal = np.dot(left_term, x_minus_mu.T)
print(mahal)

0.9007500443526901


In [73]:
# Calculate the dot product of two vectors
def dot_product(a, b):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * b[i]
        
    return sum

In [74]:
# Do matrix multiplication
def matrix_multiply(a, b):
    
    # Find number of rows and columns
    a_rows = len(a)
    a_cols = len(a[0])
    b_rows = len(b)
    b_cols = len(b[0])
    
    # Check to make sure we can multiply the matrices
    if a_cols != b_rows:
        print('Dimension mismatch')
        return [[]]
    
    # Create return matrix
    prod = [[0 for i in range(b_cols)] for j in range(a_rows)]
    
    # Calculate the dot product for each element in the return matrix
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):
                prod[i][j] += a[i][k] * b[k][j]
    
    return prod

In [75]:
# Calculate Mahalanobis distance the C way (with const inverse covariance)
def mahalanobis(x, mu, inv_cov):
    
    # Subtract each element in X from the mean
    x_minus_mu = [0 for i in range(len(x))]
    for i in range(len(x)):
        x_minus_mu[i] = x[i] - mu[i]
        
    # Compute product of prev term and inverse covariance
    left_term = matrix_multiply([x_minus_mu], inv_cov)
    
    # Transpose difference matrix
    x_minus_mu_t = [[i] for i in x_minus_mu]
    
    # Matrix multiply prev term and difference
    mahal = matrix_multiply(left_term, x_minus_mu_t)
    
    return mahal[0][0]

In [76]:
# Test Mahalanobis function with normal sample
print('Normal Sample')
print('MAD:', normal_x)
print('Mahalanobis distance:', mahalanobis(normal_x, model_mu.tolist(), inv_cov.tolist()))

Normal Sample
MAD: [0.006518250900000001, 0.011357457299999998, 0.0020734161]
Mahalanobis distance: 0.9007500443526901


In [77]:
# Test Mahalanobis function with apnea sample
print('apnea Sample')
sample = apnea_sample[0:max_measurements]  # Truncate to 128 measurements
apnea_x = extract_mad_features(sample)
print('MAD:', apnea_x)
print('Mahalanobis distance:', mahalanobis(apnea_x, model_mu, inv_cov))

apnea Sample
MAD: [0.004049721900000002, 0.0123456102, 0.0027657903]
Mahalanobis distance: 1.9775592098662484
